In [1]:
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [3]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [4]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [8]:
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return numeric results, with format (value, unit). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = str(e)
            print('[Error] ' + s)
            return None
table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
code = gen_code(llm, 'What is the change in Fixed Price in 2019 from 2018?', table)
exec_code(code, table)

(306.20000000000005, '$')

In [29]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            code = gen_code(llm, q, table)
            r = exec_code(code, table)
            if r is not None:
                if len(r) == 2:
                    (pred_value, pred_unit) = r
                elif len(r) == 1:
                    pred_value = r                
            else:
                (pred_value, pred_unit) = (None, None)
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code))


0it [00:00, ?it/s]

[Error] invalid literal for int() with base 10: '6,106'
[Error] '(' was never closed (<string>, line 2)
[Error] invalid literal for int() with base 10: '$1625687'
[Error] invalid literal for int() with base 10: '(7)'
[Error] invalid literal for int() with base 10: '(9)'
[Error] invalid literal for int() with base 10: '—'
[Error] unterminated string literal (detected at line 17) (<string>, line 17)
[Error] could not convert string to float: '(42.7)'
[Error] invalid literal for int() with base 10: ',63'
[Error] invalid literal for int() with base 10: 'Cash-settled awards'
[Error] invalid literal for int() with base 10: '2,000'
[Error] invalid literal for int() with base 10: '2,082'
[Error] division by zero
[Error] name 'Table' is not defined
[Error] '(' was never closed (<string>, line 3)
[Error] invalid literal for int() with base 10: '1,310'
[Error] '(' was never closed (<string>, line 3)
[Error] '[' was never closed (<string>, line 2)
[Error] could not convert string to float: '(1.3'


In [30]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.8076923076923077 0.8076923076923077 0.8176638176638177


In [13]:
import pickle
with open('res_p2_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [27]:
res[1]


({'answer_type': 'arithmetic', 'answer': -22.22, 'scale': 'percent'},
 None,
 'percent',
 {'uid': '05b670d3-5b19-438c-873f-9bf6de29c69e',
  'order': 6,
  'question': 'What is the percentage change in Other in 2019 from 2018?',
  'answer': -22.22,
  'derivation': '(44.1-56.7)/56.7',
  'answer_type': 'arithmetic',
  'answer_from': 'table-text',
  'rel_paragraphs': ['2'],
  'req_comparison': False,
  'scale': 'percent'},
 "def run():\n    percentage_change = ((float(44.1) - float(56.7)) / float(56.7)) * 100\n    return (round(percentage_change, 2), 'percent')")

In [20]:
len((1,2))

2